In [1]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/dependency/gsd-ud-train.conllu.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/dependency/gsd-ud-test.conllu.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/dependency/gsd-ud-dev.conllu.txt
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/dependency/augmented-dependency.json
# https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/bert-bahasa/bert-base-13-9-2019.tar.gz
# tar -zxf bert-base-13-9-2019.tar.gz

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [3]:
with open('gsd-ud-train.conllu.txt') as fopen:
    corpus = fopen.read().split('\n')
    
with open('gsd-ud-test.conllu.txt') as fopen:
    corpus.extend(fopen.read().split('\n'))
    
with open('gsd-ud-dev.conllu.txt') as fopen:
    corpus.extend(fopen.read().split('\n'))

In [4]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import tensorflow as tf
import numpy as np

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/ten

In [5]:
import unicodedata
import six
from functools import partial

SPIECE_UNDERLINE = '▁'

def preprocess_text(inputs, lower=False, remove_space=True, keep_accents=False):
  if remove_space:
    outputs = ' '.join(inputs.strip().split())
  else:
    outputs = inputs
  outputs = outputs.replace("``", '"').replace("''", '"')

  if six.PY2 and isinstance(outputs, str):
    outputs = outputs.decode('utf-8')

  if not keep_accents:
    outputs = unicodedata.normalize('NFKD', outputs)
    outputs = ''.join([c for c in outputs if not unicodedata.combining(c)])
  if lower:
    outputs = outputs.lower()

  return outputs


def encode_pieces(sp_model, text, return_unicode=True, sample=False):
  # return_unicode is used only for py2

  # note(zhiliny): in some systems, sentencepiece only accepts str for py2
  if six.PY2 and isinstance(text, unicode):
    text = text.encode('utf-8')

  if not sample:
    pieces = sp_model.EncodeAsPieces(text)
  else:
    pieces = sp_model.SampleEncodeAsPieces(text, 64, 0.1)
  new_pieces = []
  for piece in pieces:
    if len(piece) > 1 and piece[-1] == ',' and piece[-2].isdigit():
      cur_pieces = sp_model.EncodeAsPieces(
          piece[:-1].replace(SPIECE_UNDERLINE, ''))
      if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
        if len(cur_pieces[0]) == 1:
          cur_pieces = cur_pieces[1:]
        else:
          cur_pieces[0] = cur_pieces[0][1:]
      cur_pieces.append(piece[-1])
      new_pieces.extend(cur_pieces)
    else:
      new_pieces.append(piece)

  # note(zhiliny): convert back to unicode for py2
  if six.PY2 and return_unicode:
    ret_pieces = []
    for piece in new_pieces:
      if isinstance(piece, str):
        piece = piece.decode('utf-8')
      ret_pieces.append(piece)
    new_pieces = ret_pieces

  return new_pieces


def encode_ids(sp_model, text, sample=False):
  pieces = encode_pieces(sp_model, text, return_unicode=False, sample=sample)
  ids = [sp_model.PieceToId(piece) for piece in pieces]
  return ids

In [6]:
import sentencepiece as spm

sp_model = spm.SentencePieceProcessor()
sp_model.Load('bert-base/sp10m.cased.v4.model')

with open('bert-base/sp10m.cased.v4.vocab') as fopen:
    v = fopen.read().split('\n')[:-1]
v = [i.split('\t') for i in v]
v = {i[0]: i[1] for i in v}

class Tokenizer:
    def __init__(self, v):
        self.vocab = v
        pass
    
    def tokenize(self, string):
        return encode_pieces(sp_model, string, return_unicode=False, sample=False)
    
    def convert_tokens_to_ids(self, tokens):
        return [sp_model.PieceToId(piece) for piece in tokens]
    
    def convert_ids_to_tokens(self, ids):
        return [sp_model.IdToPiece(i) for i in ids]
    
tokenizer = Tokenizer(v)

In [7]:
tag2idx = {'PAD': 0, 'X': 1}
tag_idx = 2

def process_corpus(corpus, until = None):
    global word2idx, tag2idx, char2idx, word_idx, tag_idx, char_idx
    sentences, words, depends, labels, pos, sequences = [], [], [], [], [], []
    temp_sentence, temp_word, temp_depend, temp_label, temp_pos = [], [], [], [], []
    first_time = True
    for sentence in corpus:
        try:
            if len(sentence):
                if sentence[0] == '#':
                    continue
                if first_time:
                    print(sentence)
                    first_time = False
                sentence = sentence.split('\t')
                if sentence[7] not in tag2idx:
                    tag2idx[sentence[7]] = tag_idx
                    tag_idx += 1
                temp_word.append(sentence[1])
                temp_depend.append(int(sentence[6]) + 1)
                temp_label.append(tag2idx[sentence[7]])
                temp_sentence.append(sentence[1])
                temp_pos.append(sentence[3])
            else:
                if len(temp_sentence) < 2 or len(temp_word) != len(temp_label):
                    temp_word = []
                    temp_depend = []
                    temp_label = []
                    temp_sentence = []
                    temp_pos = []
                    continue
                bert_tokens = ['<cls>']
                labels_ = [0]
                depends_ = [0]
                seq_ = []
                for no, orig_token in enumerate(temp_word):
                    labels_.append(temp_label[no])
                    depends_.append(temp_depend[no])
                    t = tokenizer.tokenize(orig_token)
                    bert_tokens.extend(t)
                    labels_.extend([1] * (len(t) - 1))
                    depends_.extend([0] * (len(t) - 1))
                    seq_.append(no + 1)
                bert_tokens.append('<sep>')
                labels_.append(0)
                depends_.append(0)
                words.append(tokenizer.convert_tokens_to_ids(bert_tokens))
                depends.append(depends_)
                labels.append(labels_)
                sentences.append(bert_tokens)
                pos.append(temp_pos)
                sequences.append(seq_)
                temp_word = []
                temp_depend = []
                temp_label = []
                temp_sentence = []
                temp_pos = []
        except Exception as e:
            print(e, sentence)
    return sentences[:-1], words[:-1], depends[:-1], labels[:-1], pos[:-1], sequences[:-1]

In [8]:
sentences, words, depends, labels, _, _ = process_corpus(corpus)

1	Sembungan	sembungan	PROPN	X--	_	4	nsubj	_	MorphInd=^sembungan<x>_X--$


In [9]:
list(zip(sentences[0], depends[0]))

[('<cls>', 0),
 ('▁S', 5),
 ('embung', 0),
 ('an', 0),
 ('▁adalah', 5),
 ('▁sebuah', 5),
 ('▁desa', 1),
 ('▁yang', 7),
 ('▁terletak', 5),
 ('▁di', 9),
 ('▁kecamatan', 7),
 ('▁Ke', 9),
 ('jajar', 0),
 ('▁', 9),
 (',', 0),
 ('▁kabupaten', 9),
 ('▁Won', 12),
 ('oso', 0),
 ('bo', 0),
 ('▁', 12),
 (',', 0),
 ('▁Jawa', 12),
 ('▁Tengah', 15),
 ('▁', 15),
 (',', 0),
 ('▁Indonesia', 15),
 ('▁', 5),
 ('.', 0),
 ('<sep>', 0)]

In [10]:
import json

with open('augmented-dependency.json') as fopen:
    augmented = json.load(fopen)

In [11]:
text_augmented, depends_augmented, labels_augmented = [], [], []

for a in augmented:
    text_augmented.extend(a[0])
    depends_augmented.extend(a[1])
    labels_augmented.extend((np.array(a[2]) + 1).tolist())

In [12]:
text_augmented[0], depends_augmented[0], labels_augmented[0]

('Sembungan termasuklah sebuah desa yang terletak di kecamatan Kejajar Empty kabupaten Sukoharjo Elegant Awa Wittes Fake Zaryadye Considerable',
 [5, 5, 5, 1, 7, 5, 9, 7, 9, 9, 9, 12, 12, 12, 15, 15, 15, 5],
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 10, 11, 12, 13, 11, 12, 11])

In [13]:
def parse_XY(texts, depends, labels):
    outside, sentences, outside_depends, outside_labels = [], [], [], []
    for no, text in enumerate(texts):
        temp_depend = depends[no]
        temp_label = labels[no]
        s = text.split()
        sentences.append(s)
        bert_tokens = ['<cls>']
        labels_ = [0]
        depends_ = [0]
        for no, orig_token in enumerate(s):
            labels_.append(temp_label[no])
            depends_.append(temp_depend[no])
            t = tokenizer.tokenize(orig_token)
            bert_tokens.extend(t)
            labels_.extend([1] * (len(t) - 1))
            depends_.extend([0] * (len(t) - 1))
        bert_tokens.append('<sep>')
        labels_.append(0)
        depends_.append(0)
        outside.append(tokenizer.convert_tokens_to_ids(bert_tokens))
        outside_depends.append(depends_)
        outside_labels.append(labels_)
    return outside, sentences, outside_depends, outside_labels

In [14]:
outside, _, outside_depends, outside_labels = parse_XY(text_augmented, 
                                                       depends_augmented, 
                                                       labels_augmented)

In [15]:
words.extend(outside)
depends.extend(outside_depends)
labels.extend(outside_labels)

In [16]:
idx2tag = {v:k for k, v in tag2idx.items()}

In [17]:
from sklearn.model_selection import train_test_split

words_train, words_test, depends_train, depends_test, labels_train, labels_test \
= train_test_split(words, depends, labels, test_size = 0.2)

In [18]:
len(words_train), len(words_test)

(40289, 10073)

In [19]:
train_X = words_train
train_Y = labels_train
train_depends = depends_train

test_X = words_test
test_Y = labels_test
test_depends = depends_test

In [20]:
epoch = 10
batch_size = 32
warmup_proportion = 0.1
num_train_steps = int(len(train_X) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)
bert_config = modeling.BertConfig.from_json_file('bert-base/bert_config.json')

In [21]:
class BiAAttention:
    def __init__(self, input_size_encoder, input_size_decoder, num_labels):
        self.input_size_encoder = input_size_encoder
        self.input_size_decoder = input_size_decoder
        self.num_labels = num_labels
        
        self.W_d = tf.get_variable("W_d", shape=[self.num_labels, self.input_size_decoder],
           initializer=tf.contrib.layers.xavier_initializer())
        self.W_e = tf.get_variable("W_e", shape=[self.num_labels, self.input_size_encoder],
           initializer=tf.contrib.layers.xavier_initializer())
        self.U = tf.get_variable("U", shape=[self.num_labels, self.input_size_decoder, self.input_size_encoder],
           initializer=tf.contrib.layers.xavier_initializer())
        
    def forward(self, input_d, input_e, mask_d=None, mask_e=None):
        batch = tf.shape(input_d)[0]
        length_decoder = tf.shape(input_d)[1]
        length_encoder = tf.shape(input_e)[1]
        out_d = tf.expand_dims(tf.matmul(self.W_d, tf.transpose(input_d, [0, 2, 1])), 3)
        out_e = tf.expand_dims(tf.matmul(self.W_e, tf.transpose(input_e, [0, 2, 1])), 2)
        output = tf.matmul(tf.expand_dims(input_d, 1), self.U)
        output = tf.matmul(output, tf.transpose(tf.expand_dims(input_e, 1), [0, 1, 3, 2]))
        
        output = output + out_d + out_e
        
        if mask_d is not None:
            d = tf.expand_dims(tf.expand_dims(mask_d, 1), 3)
            e = tf.expand_dims(tf.expand_dims(mask_e, 1), 2)
            output = output * d * e
            
        return output

class Model:
    def __init__(
        self,
        learning_rate,
        hidden_size_word,
    ):
        def cells(size, reuse = False):
            return tf.contrib.rnn.DropoutWrapper(
                tf.nn.rnn_cell.LSTMCell(
                    size,
                    initializer = tf.orthogonal_initializer(),
                    reuse = reuse,
                ),
                output_keep_prob = dropout,
            )
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.labels = tf.placeholder(tf.int32, shape = [None, None])
        self.depends = tf.placeholder(tf.int32, shape = [None, None])
        self.maxlen = tf.shape(self.X)[1]
        self.lengths = tf.count_nonzero(self.X, 1)
        self.mask = tf.math.not_equal(self.X, 0)
        float_mask = tf.cast(self.mask, tf.float32)
        
        self.arc_h = tf.layers.Dense(hidden_size_word)
        self.arc_c = tf.layers.Dense(hidden_size_word)
        self.attention = BiAAttention(hidden_size_word, hidden_size_word, 1)

        model = modeling.BertModel(
            config=bert_config,
            is_training=True,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        output_layer = model.get_sequence_output()

        logits = tf.layers.dense(output_layer, len(idx2tag))
        log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(
            logits, self.labels, self.lengths
        )
        arc_h = tf.nn.elu(self.arc_h(output_layer))
        arc_c = tf.nn.elu(self.arc_c(output_layer))
        out_arc = tf.squeeze(self.attention.forward(arc_h, arc_h, mask_d=float_mask, mask_e=float_mask), axis = 1)
        
        batch = tf.shape(out_arc)[0]
        batch_index = tf.range(0, batch)
        max_len = tf.shape(out_arc)[1]
        sec_max_len = tf.shape(out_arc)[2]
        
        minus_inf = -1e8
        minus_mask = (1 - float_mask) * minus_inf
        out_arc = out_arc + tf.expand_dims(minus_mask, axis = 2) + tf.expand_dims(minus_mask, axis = 1)
        loss_arc = tf.nn.log_softmax(out_arc, dim=1)
        loss_arc = loss_arc * tf.expand_dims(float_mask, axis = 2) * tf.expand_dims(float_mask, axis = 1)
        num = tf.reduce_sum(float_mask) - tf.cast(batch, tf.float32)
        
        child_index = tf.tile(tf.expand_dims(tf.range(0, max_len), 1), [1, batch])
        t = tf.transpose(self.depends)
        broadcasted = tf.broadcast_to(batch_index, tf.shape(t))
        concatenated = tf.transpose(tf.concat([tf.expand_dims(broadcasted, axis = 0),
                                               tf.expand_dims(t, axis = 0),
                                               tf.expand_dims(child_index, axis = 0)], axis = 0))
        loss_arc = tf.gather_nd(loss_arc, concatenated)
        loss_arc = tf.transpose(loss_arc, [1, 0])[1:]
        
        loss_arc = tf.reduce_sum(-loss_arc) / num
        
        self.cost = tf.reduce_mean(-log_likelihood) + loss_arc
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        
        mask = tf.sequence_mask(self.lengths, maxlen = self.maxlen)
        
        self.tags_seq, _ = tf.contrib.crf.crf_decode(
            logits, transition_params, self.lengths
        )
        
        out_arc = out_arc + tf.linalg.diag(tf.fill([max_len], -np.inf))
        minus_mask = tf.expand_dims(tf.cast(1.0 - float_mask, tf.bool), axis = 2)
        minus_mask = tf.tile(minus_mask, [1, 1, sec_max_len])
        out_arc = tf.where(minus_mask, tf.fill(tf.shape(out_arc), -np.inf), out_arc)
        self.heads = tf.argmax(out_arc, axis = 1)
        
        self.prediction = tf.boolean_mask(self.tags_seq, mask)
        mask_label = tf.boolean_mask(self.labels, mask)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        self.prediction = tf.cast(tf.boolean_mask(self.heads, mask), tf.int32)
        mask_label = tf.boolean_mask(self.depends, mask)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy_depends = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [22]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

hidden_size_word = 128
learning_rate = 2e-5

model = Model(learning_rate,hidden_size_word)
sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
dim is deprecated, use axis instead


Instructi

In [23]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, 'bert-base/model.ckpt')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from bert-base/model.ckpt


In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

batch_x = train_X[:5]
batch_x = pad_sequences(batch_x,padding='post')
batch_y = train_Y[:5]
batch_y = pad_sequences(batch_y,padding='post')
batch_depends = train_depends[:5]
batch_depends = pad_sequences(batch_depends,padding='post')

In [25]:
sess.run([model.accuracy, model.accuracy_depends, model.cost],
        feed_dict = {model.X: batch_x,
                model.labels: batch_y,
                model.depends: batch_depends})

[0.0047393367, 0.023696683, 212.74112]

In [26]:
from tqdm import tqdm

for e in range(epoch):
    train_acc, train_loss = [], []
    test_acc, test_loss = [], []
    train_acc_depends, test_acc_depends = [], []
    
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = train_X[i: index]
        batch_x = pad_sequences(batch_x,padding='post')
        batch_y = train_Y[i: index]
        batch_y = pad_sequences(batch_y,padding='post')
        batch_depends = train_depends[i: index]
        batch_depends = pad_sequences(batch_depends,padding='post')
        
        acc_depends, acc, cost, _ = sess.run(
            [model.accuracy_depends, model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.X: batch_x,
                model.labels: batch_y,
                model.depends: batch_depends
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        train_acc_depends.append(acc_depends)
        pbar.set_postfix(cost = cost, accuracy = acc, accuracy_depends = acc_depends)
        
    pbar = tqdm(
        range(0, len(test_X), batch_size), desc = 'test minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = test_X[i: index]
        batch_x = pad_sequences(batch_x,padding='post')
        batch_y = test_Y[i: index]
        batch_y = pad_sequences(batch_y,padding='post')
        batch_depends = test_depends[i: index]
        batch_depends = pad_sequences(batch_depends,padding='post')
        
        acc_depends, acc, cost = sess.run(
            [model.accuracy_depends, model.accuracy, model.cost],
            feed_dict = {
                model.X: batch_x,
                model.labels: batch_y,
                model.depends: batch_depends
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        test_acc_depends.append(acc_depends)
        pbar.set_postfix(cost = cost, accuracy = acc, accuracy_depends = acc_depends)
    
    
    print(
    'epoch: %d, training loss: %f, training acc: %f, training depends: %f, valid loss: %f, valid acc: %f, valid depends: %f\n'
    % (e, np.mean(train_loss), 
       np.mean(train_acc), 
       np.mean(train_acc_depends), 
       np.mean(test_loss), 
       np.mean(test_acc), 
       np.mean(test_acc_depends)
    ))

train minibatch loop:   0%|          | 0/1260 [00:00<?, ?it/s]

epoch: 0, training loss: 24.551825, training acc: 0.846269, training depends: 0.456185, valid loss: 13.363911, valid acc: 0.909736, valid depends: 0.502892



train minibatch loop:   0%|          | 0/1260 [00:00<?, ?it/s]

epoch: 1, training loss: 10.271261, training acc: 0.931256, training depends: 0.515334, valid loss: 9.201717, valid acc: 0.939426, valid depends: 0.522873



train minibatch loop:   0%|          | 0/1260 [00:00<?, ?it/s]

...

epoch: 7, training loss: 1.822270, training acc: 0.994996, training depends: 0.585704, valid loss: 4.610699, valid acc: 0.977195, valid depends: 0.579058



train minibatch loop:   0%|          | 0/1260 [00:00<?, ?it/s]

epoch: 8, training loss: 1.652127, training acc: 0.996198, training depends: 0.592138, valid loss: 4.524382, valid acc: 0.977961, valid depends: 0.583688



test minibatch loop: 100%|██████████| 315/315 [01:20<00:00,  3.91it/s, accuracy=0.979, accuracy_depends=0.593, cost=5.23]

epoch: 9, training loss: 1.539244, training acc: 0.996957, training depends: 0.596063, valid loss: 4.426018, valid acc: 0.978670, valid depends: 0.585346



In [27]:
def evaluate(heads_pred, types_pred, heads, types, lengths,
             symbolic_root=False, symbolic_end=False):
    batch_size, _ = heads_pred.shape
    ucorr = 0.
    lcorr = 0.
    total = 0.
    ucomplete_match = 0.
    lcomplete_match = 0.

    corr_root = 0.
    total_root = 0.
    start = 1 if symbolic_root else 0
    end = 1 if symbolic_end else 0
    for i in range(batch_size):
        ucm = 1.
        lcm = 1.
        for j in range(start, lengths[i] - end):

            total += 1
            if heads[i, j] == heads_pred[i, j]:
                ucorr += 1
                if types[i, j] == types_pred[i, j]:
                    lcorr += 1
                else:
                    lcm = 0
            else:
                ucm = 0
                lcm = 0

            if heads[i, j] == 0:
                total_root += 1
                corr_root += 1 if heads_pred[i, j] == 0 else 0

        ucomplete_match += ucm
        lcomplete_match += lcm
    
    return ucorr / total, lcorr / total, corr_root / total_root

In [28]:
tags_seq, heads = sess.run(
    [model.tags_seq, model.heads],
    feed_dict = {
        model.X: batch_x,
    },
)

# do not forget minus by 1, because we plus by 1 during data processing to differentiate BERT padding
tags_seq[0], heads[0] - 1, batch_depends[0] - 1

(array([ 0,  8,  1,  1,  9,  1, 22, 22, 22, 11,  1,  1, 22, 11,  1, 12,  1,
         1, 11,  1, 14, 10,  1,  1, 11,  1,  1,  2, 10,  1, 11,  1,  1, 12,
         1, 10,  1, 10,  5,  7,  8, 21,  1, 11,  1,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32),
 array([41,  3, -1, -1, 11, -1,  3,  4,  4,  5, -1, -1,  6,  7, -1,  7, -1,
        -1, 10, -1,  7, 12, -1, -1, 13, -1, -1, 15, 15, -1, 15, -1, -1, 15,
        -1, 19, -1, 14,  

In [29]:
arc_accuracy, type_accuracy, root_accuracy = evaluate(heads, tags_seq, batch_depends, batch_y, 
        np.count_nonzero(batch_x, axis = 1))
arc_accuracy, type_accuracy, root_accuracy

(0.5878378378378378, 0.583976833976834, 0.94824016563147)

In [30]:
arcs, types, roots = [], [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'test minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_X))
    batch_x = test_X[i: index]
    batch_x = pad_sequences(batch_x,padding='post')
    batch_y = test_Y[i: index]
    batch_y = pad_sequences(batch_y,padding='post')
    batch_depends = test_depends[i: index]
    batch_depends = pad_sequences(batch_depends,padding='post')
    
    tags_seq, heads = sess.run(
        [model.tags_seq, model.heads],
        feed_dict = {
            model.X: batch_x
        },
    )
    
    arc_accuracy, type_accuracy, root_accuracy = evaluate(heads - 1, tags_seq, batch_depends - 1, batch_y, 
            np.count_nonzero(batch_x, axis = 1))
    pbar.set_postfix(arc_accuracy = arc_accuracy, type_accuracy = type_accuracy, 
                     root_accuracy = root_accuracy)
    arcs.append(arc_accuracy)
    types.append(type_accuracy)
    roots.append(root_accuracy)

test minibatch loop: 100%|██████████| 315/315 [01:18<00:00,  3.99it/s, arc_accuracy=0.589, root_accuracy=0.88, type_accuracy=0.585] 


In [31]:
print('arc accuracy:', np.mean(arcs))
print('types accuracy:', np.mean(types))
print('root accuracy:', np.mean(roots))

arc accuracy: 0.5855140876299667
types accuracy: 0.5812892180763379
root accuracy: 0.8887063492063492
